## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import EPKADataset, load_partition,save_partition, create_train_valid_loaders
from torch.utils.data import DataLoader

from purrfect.training import train_model,train_validate
import torch.optim as optim

from purrfect.metrics import MetricAccumulator
from purrfect.active_learning import create_new_partition,create_next_partitions, test_model

from sklearn.model_selection import train_test_split
from purrfect.submission import create_submission

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Definición modelo

In [4]:
# Define the double convolution block
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class ChannelAdder(nn.Module):
    def __init__(self):
        super(ChannelAdder, self).__init__()
        # Define Sobel and Laplacian kernels as tensors
        self.sobel_x = torch.tensor([[-1., 0., 1.],
                                     [-2., 0., 2.],
                                     [-1., 0., 1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)
        
        self.sobel_y = torch.tensor([[-1., -2., -1.],
                                     [ 0.,  0.,  0.],
                                     [ 1.,  2.,  1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)
        
        self.laplacian_kernel = torch.tensor([[0.,  1., 0.],
                                              [1., -4., 1.],
                                              [0.,  1., 0.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)

    def forward(self, x):
        epsilon = 1e-8

        # Extract the first, second, and third channels
        first_channel = x[:, 0, :, :].unsqueeze(1)  # e1 (first channel)
        second_channel = x[:, 1, :, :].unsqueeze(1)  # e2 (second channel)
        third_channel = x[:, 2, :, :].unsqueeze(1)  # error (third channel)

        # 1. Compute the first new channel: sqrt(first_channel^2 + second_channel^2)
        new_channel1 = torch.sqrt(first_channel**2 + second_channel**2)

        # 2. Compute the second new channel: 1/2 * arctan(channel2 / channel1)
        new_channel2 = 0.5 * torch.atan(second_channel / (first_channel + epsilon))

        # 3. Compute Sobel gradients and Laplacians for e1 (first_channel)
        grad_e1_x = F.conv2d(first_channel, self.sobel_x, padding=1)
        grad_e1_y = F.conv2d(first_channel, self.sobel_y, padding=1)
        grad_e1_magnitude = torch.sqrt(grad_e1_x**2 + grad_e1_y**2)
        #laplacian_e1 = F.conv2d(first_channel, self.laplacian_kernel, padding=1)

        # 4. Compute Sobel gradients and Laplacians for e2 (second_channel)
        grad_e2_x = F.conv2d(second_channel, self.sobel_x, padding=1)
        grad_e2_y = F.conv2d(second_channel, self.sobel_y, padding=1)
        grad_e2_magnitude = torch.sqrt(grad_e2_x**2 + grad_e2_y**2)
        #laplacian_e2 = F.conv2d(second_channel, self.laplacian_kernel, padding=1)

        # 5. Compute weighted ellipticity channels (e1_weighted, e2_weighted)
        e1_weighted = first_channel / (third_channel + epsilon)
        e2_weighted = second_channel / (third_channel + epsilon)

        # Concatenate all the channels (original and new) into the output tensor
        output = torch.cat([
            x,                 # Original 3 channels
            new_channel1,      # sqrt(channel1^2 + channel2^2)
            new_channel2,      # 1/2 * arctan(channel2 / channel1)
            #grad_e1_x,         # Gradient X of channel1
            #grad_e1_y,         # Gradient Y of channel1
            #grad_e2_x,         # Gradient X of channel2
            #grad_e2_y,         # Gradient Y of channel2
            grad_e1_magnitude, # Gradient magnitude of channel1
            grad_e2_magnitude, # Gradient magnitude of channel2
            #laplacian_e1,      # Laplacian of channel1
            #laplacian_e2,      # Laplacian of channel2
            e1_weighted,       # e1_weighted
            e2_weighted        # e2_weighted
        ], dim=1)

        return output

# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        # Channel Adder
        self.channel_adder = ChannelAdder()
        # Encoder
        self.encoder1 = DoubleConv(in_channels+6, 64)
        self.encoder2 = DoubleConv(64, 128)
        self.encoder3 = DoubleConv(128, 256)
        self.encoder4 = DoubleConv(256, 512)

        # Bottleneck
        self.bottleneck = DoubleConv(512, 1024)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = DoubleConv(128, 64)

        # Final output layer
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Agregar features
        x = self.channel_adder(x)
        
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(F.max_pool2d(e1, 2))
        e3 = self.encoder3(F.max_pool2d(e2, 2))
        e4 = self.encoder4(F.max_pool2d(e3, 2))

        # Bottleneck
        b = self.bottleneck(F.max_pool2d(e4, 2))

        # Decoder with skip connections
        d4 = self.upconv4(b)
        d4 = torch.cat((e4, d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((e3, d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((e2, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((e1, d1), dim=1)
        d1 = self.decoder1(d1)

        # Final output
        out = self.final_conv(d1)
        return out

## Creación particion inicial

In [5]:
#Creacion de particiones train y valid
init_partition = load_partition("partition_1.json")
train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
save_partition("partition_1_train.json","partitions",train_partition)
save_partition("partition_1_val.json","partitions",val_partition)

In [22]:
def create_next_partitions(current_partition):#Creacion de particiones train y valid
    init_partition = load_partition(f"partition_{current_partition+1}.json")
    train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
    save_partition(f"partition_{current_partition+1}_train.json","partitions",train_partition)
    save_partition(f"partition_{current_partition+1}_val.json","partitions",val_partition)

## Carga modelo

In [5]:
#Define model
model = UNet( 3, 1)
model = model.to(DEVICE)
# Define Loss
criterion = torch.nn.L1Loss()

In [6]:
sum([p.numel() for p in model.parameters() if p.requires_grad])

31046977

In [10]:
current_partition = 1
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)

In [8]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:11<00:00,  6.53it/s, WMAPE=4.17, DICE=0.131, DPEAKS=94.6, Loss=0.013]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.55it/s, WMAPE=2.37, DICE=0.103, DPEAKS=87.9, Loss=0.00815]


Saving best model
Epoch [3/50]


...

Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:11<00:00,  6.57it/s, WMAPE=5.81, DICE=0.215, DPEAKS=144, Loss=0.022]


Epoch [7/50]


Validate Epoch 7: 100%|██████████| 77/77 [00:10<00:00,  7.06it/s, WMAPE=3.11, DICE=0.074, DPEAKS=61.3, Loss=0.00956]


early stopping: 3 epochs without improvement
Training complete.


In [9]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: 100%|██████████| 1941/1941 [04:42<00:00,  6.87it/s, WMAPE=1.91, DICE=0.0763, DPEAKS=59.1, Loss=0.00697]


In [13]:
#Creacion de particiones train y valid
init_partition = load_partition(f"partition_{current_partition+1}.json")
train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
save_partition(f"partition_{current_partition+1}_train.json","partitions",train_partition)
save_partition(f"partition_{current_partition+1}_val.json","partitions",val_partition)

In [14]:
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
model.load_state_dict(torch.load(best_model_path,weights_only=True))
current_partition=2

In [15]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  6.31it/s, WMAPE=2.71, DICE=0.0984, DPEAKS=92.5, Loss=0.00977]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.89it/s, WMAPE=2.54, DICE=0.0732, DPEAKS=66.3, Loss=0.00774]


Saving best model
Epoch [3/50]


...

Epoch [8/50]


Validate Epoch 8: 100%|██████████| 77/77 [00:11<00:00,  6.86it/s, WMAPE=1.76, DICE=0.0614, DPEAKS=50, Loss=0.00581]


Epoch [9/50]


Validate Epoch 9: 100%|██████████| 77/77 [00:11<00:00,  6.71it/s, WMAPE=1.93, DICE=0.0635, DPEAKS=58.6, Loss=0.00591]


early stopping: 3 epochs without improvement
Training complete.


In [16]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: 100%|██████████| 1941/1941 [05:14<00:00,  6.17it/s, WMAPE=1.83, DICE=0.0582, DPEAKS=49, Loss=0.0054]


In [18]:
create_new_partition(current_partition)

In [19]:
current_partition=3
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)

In [20]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  5.96it/s, WMAPE=2.03, DICE=0.0603, DPEAKS=61.1, Loss=0.00608]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:10<00:00,  7.13it/s, WMAPE=5.36, DICE=0.0579, DPEAKS=45, Loss=0.0157]


Epoch [3/50]


...

Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:10<00:00,  7.17it/s, WMAPE=2.04, DICE=0.0634, DPEAKS=56.6, Loss=0.00611]


Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:10<00:00,  7.17it/s, WMAPE=2.73, DICE=0.0758, DPEAKS=71.2, Loss=0.0079]


early stopping: 3 epochs without improvement
Training complete.


In [21]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: 100%|██████████| 1941/1941 [04:54<00:00,  6.59it/s, WMAPE=1.77, DICE=0.0499, DPEAKS=38.4, Loss=0.00553]


TypeError: create_next_partitions() missing 2 required positional arguments: 'model' and 'criterion'

In [23]:
create_next_partitions(current_partition)

In [24]:
current_partition=4

In [25]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  6.06it/s, WMAPE=1.51, DICE=0.0393, DPEAKS=27.1, Loss=0.00439]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.91it/s, WMAPE=1.72, DICE=0.0389, DPEAKS=28.4, Loss=0.00493]


Epoch [3/50]


...

Epoch [8/50]


Validate Epoch 8: 100%|██████████| 77/77 [00:10<00:00,  7.07it/s, WMAPE=1.45, DICE=0.0379, DPEAKS=27.2, Loss=0.00424]


Epoch [9/50]


Validate Epoch 9: 100%|██████████| 77/77 [00:11<00:00,  6.50it/s, WMAPE=1.41, DICE=0.038, DPEAKS=26.9, Loss=0.00424]


early stopping: 3 epochs without improvement
Training complete.


In [26]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition)

Validate Epoch test: 100%|██████████| 1941/1941 [04:50<00:00,  6.68it/s, WMAPE=1.41, DICE=0.0383, DPEAKS=28.1, Loss=0.00421]


In [27]:
current_partition=5

In [10]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:09<00:00,  7.86it/s, WMAPE=1.87, DICE=0.0453, DPEAKS=46.4, Loss=0.00513]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:10<00:00,  7.59it/s, WMAPE=1.97, DICE=0.0453, DPEAKS=44, Loss=0.00521]


Epoch [3/50]


Validate Epoch 3: 100%|██████████| 77/77 [00:09<00:00,  8.01it/s, WMAPE=2.45, DICE=0.0461, DPEAKS=44, Loss=0.00671]


Epoch [4/50]


Validate Epoch 4: 100%|██████████| 77/77 [00:09<00:00,  7.87it/s, WMAPE=1.92, DICE=0.0455, DPEAKS=45.8, Loss=0.00514]


early stopping: 3 epochs without improvement
Training complete.


In [11]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_6_val: 100%|██████████| 77/77 [00:10<00:00,  7.21it/s, WMAPE=1.65, DICE=0.0436, DPEAKS=36.1, Loss=0.00477]


In [12]:
current_partition=6

In [13]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:09<00:00,  8.14it/s, WMAPE=2.05, DICE=0.046, DPEAKS=52.4, Loss=0.00538]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:09<00:00,  8.04it/s, WMAPE=2.06, DICE=0.0462, DPEAKS=52.8, Loss=0.00534]


Saving best model
Epoch [3/50]


...

Epoch [14/50]


Validate Epoch 14: 100%|██████████| 77/77 [00:09<00:00,  8.05it/s, WMAPE=2.14, DICE=0.0452, DPEAKS=49.8, Loss=0.00559]


Epoch [15/50]


Validate Epoch 15: 100%|██████████| 77/77 [00:09<00:00,  8.12it/s, WMAPE=2.04, DICE=0.046, DPEAKS=50.1, Loss=0.00524]


early stopping: 3 epochs without improvement
Training complete.


In [14]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_7_val: 100%|██████████| 77/77 [00:10<00:00,  7.23it/s, WMAPE=1.54, DICE=0.043, DPEAKS=33.5, Loss=0.00457]


In [7]:
current_partition=4
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
model.load_state_dict(torch.load(best_model_path,weights_only=True))
create_submission(model, "partition_4_bug_op2",device=DEVICE)


Create submission total its 76522:  49%|████▉     | 37372/76522 [06:57<07:42, 84.60it/s]

In [6]:
from purrfect.dataset import TestDataset

In [7]:
test_dataset = 

In [14]:
for names,batch in test_loader:
    print(a)
    print(b.shape)
    break
def create_submission_v2(model, submission_name,submission_path="submissions",device="cuda",batch_size=16):
  model.eval()
  #crea carpeta submission/submission_name/purrfectpredict
  path = os.path.join(submission_path,submission_name,"purrfectpredict")
  os.makedirs(path, exist_ok=True)
  
  test_loader = DataLoader(TestDataset(), batch_size=batch_size, shuffle=False)
  loop = tqdm(test_loader, desc=f'Create submission total its {len(test_loader)}', leave=True)
  #evalua el modelo y crea un archivo .npy por cada prediccion del modelo, el nombre del archivo .npy debe ser el mismo que del archivo de input
  with torch.no_grad():
    for names,batch in loop:
      output = model(batch.to(device))
      for index,name in enumerate(names): 
        np.save(f"{path}/{name}", output[index].detach().cpu().numpy().astype(np.float16))

('00923_00001.npy', '00923_00002.npy', '00923_00003.npy', '00923_00004.npy', '00923_00005.npy', '00923_00006.npy', '00923_00007.npy', '00923_00008.npy', '00923_00009.npy', '00923_00010.npy', '00923_00011.npy', '00923_00012.npy', '00923_00013.npy', '00923_00014.npy', '00923_00015.npy', '00923_00016.npy')
torch.Size([16, 3, 128, 128])
